This notebook is my implementation of the Neural Collabrative Filtering Recoomendation system. 
[NCF](https://arxiv.org/abs/1708.05031) models has other implementation, such as the [author's original implementation in theano](https://github.com/hexiangnan/neural_collaborative_filtering), the [offical tensorflow/keras version](https://github.com/tensorflow/models/tree/master/official/recommendation) and a [pytorch version](
https://towardsdatascience.com/recotour-ii-neural-recommendation-algorithms-49733938d56e) with a [video](https://www.youtube.com/watch?v=O4lk9Lw7lS0) explaining it. In this notebook, I write the code so that it can be followed in a single google colab notebook with step by step instructuion

In [ ]:
# !kill -9 -1xw

### 1. Download data

In [ ]:
!curl -O 'http://files.grouplens.org/datasets/movielens/ml-100k.zip'
!unzip ml-100k.zip
!mkdir -p data

!cp ml-100k/u.data data/

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os
os.getcwd()

In [ ]:
input_file = 'data/u.data'
headers = ['user_id', 'item_id', 'rating', 'timestamp']
header_row = None
ratings_df = pd.read_csv(input_file,
                         sep='\t',
                         names=headers,
                         header=header_row,
                         dtype={
                           'user_id': np.int32,
                           'item_id': np.int32,
                           'rating': np.float32,
                           'timestamp': np.int32,
                         })

In [ ]:
ratings_df.head()

### 2. Preprocess the data

In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [ ]:
user_enc = LabelEncoder()
product_enc = LabelEncoder()
scaler = MinMaxScaler()

ratings_df['user_id'] = user_enc.fit_transform(ratings_df['user_id'].values)
ratings_df['item_id'] = product_enc.fit_transform(ratings_df['item_id'].values)
ratings_df['rating'] = scaler.fit_transform(ratings_df['rating'].values.reshape(-1, 1))

In [ ]:
ratings_df.head()

In [ ]:
n_users = len(ratings_df['user_id'].unique())
n_items = len(ratings_df['item_id'].unique())

In [ ]:
print('number of unique user {}'.format(n_users))
print('number of unique movie {}'.format(n_items))

In [ ]:
ratings_df['user_id'].min() # make sure index start from 0

In [ ]:
ratings = ratings_df[['user_id','item_id','rating']].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

tr_ratings, ts_ratings = train_test_split(ratings, test_size = 0.1)
X_train = [[tr_ratings[:,0],tr_ratings[:,1]]]
y_train = tr_ratings[:,2].reshape(-1,1)

X_test = [[ts_ratings[:,0],ts_ratings[:,1]]]
y_test = ts_ratings[:,2].reshape(-1,1)

In [ ]:
## convert the rating matrix to a sparse matrix

# from scipy.sparse import coo_matrix
# u_tr, i_tr, r_tr = zip(*tr_ratings)
# tr_sparse = coo_matrix((r_tr, (u_tr, i_tr)), shape=(n_users, n_items))

# u_ts, i_ts, r_ts = zip(*ts_ratings)
# test_sparse = coo_matrix((r_ts, (u_ts, i_ts)), shape=(n_users, n_items))

### 3. build the model

Note that the neural collaborative filtering (NCF) model is a combination of the generalized matrix factorization model(GMF) and the multilayer perception model(MLP). We can build each model first and then concatenate them. In fact, in the original paper, it shows that by pretraining each model then used the pretrained weight as initialiation point for the final model, you can reach a higher accuracy. 
![](https://drive.google.com/uc?id=1RqXBiGTCPFRpSkcTfpB7WgpYOXZyuXhx)

In [ ]:
import tensorflow as tf
print(tf.__version__)

**Part 1, build the GMF model**

In [ ]:
def build_GMF(embedding_dim, n_users, n_items):
  # Input layers
  user_input = tf.keras.layers.Input(shape=(1,)) # user
  product_input = tf.keras.layers.Input(shape=(1,)) # movie
  # Embedding layers
  User_embedding = tf.keras.layers.Embedding(n_users, embedding_dim,\
                                             embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(user_input)
  Product_embedding = tf.keras.layers.Embedding(n_items, embedding_dim, \
                                                embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(product_input)
  # Dot products
  u = tf.keras.layers.Reshape((embedding_dim,))(User_embedding)
  v = tf.keras.layers.Reshape((embedding_dim,))(Product_embedding)
  s = tf.keras.layers.Dot(axes=1)([u, v])
  model = tf.keras.Model(inputs=[user_input, product_input], outputs=s)
  print(model.summary())
  return(model)


In [ ]:
embedding_dim = 10
GMF_model = build_GMF(embedding_dim, n_users, n_items)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [ ]:
opt = tf.keras.optimizers.Adagrad(learning_rate=0.5)  #Use Adagrad to get faster learning rate
# opt = tf.keras.optimizers.Adam(learning_rate=0.01)  

GMF_model.compile(loss='mean_squared_error', optimizer=opt)

history = GMF_model.fit(x=X_train, y=y_train,
                  batch_size=64, epochs=500,
                  validation_data= (X_test, y_test),
                  callbacks=[checkpoint_callback, es])

In [ ]:
import matplotlib.pyplot as plt

def plot_graph(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [ ]:
plot_graph(history, "loss")

As we can see, we should use the model somewhere around the 10th epoches, we can load the saved weight from checkpoint file

In [ ]:
recovered_model = build_GMF(embedding_dim, n_users, n_items)
recovered_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
recovered_model.build(tf.TensorShape([1, None]))

In [ ]:
prediction = recovered_model(X_test)

In [ ]:
plt.plot(y_test,prediction,'o')
plt.xlim(-0.5, 1.5)
plt.ylim(-0.5, 1.5)
plt.gca().set_aspect('equal', adjustable='box')

One can also add a bias layer, which represent the average score for each users, and the average score for each item

In [ ]:
def build_bias(embedding_dim, n_users, n_items):
    # Input layers
    user_input = tf.keras.layers.Input(shape=(1,)) # user
    product_input = tf.keras.layers.Input(shape=(1,)) # movie
    # Embedding layers
    User_embedding = tf.keras.layers.Embedding(n_users, embedding_dim,\
                                             embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(user_input)

    User_embedding_avg = tf.keras.layers.Embedding(n_users, 1,embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(user_input)


    Product_embedding = tf.keras.layers.Embedding(n_items, embedding_dim,\
                                             embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(product_input)

    Product_embedding_avg = tf.keras.layers.Embedding(n_items, 1,embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(product_input)


    # Dot products
    u = tf.keras.layers.Reshape((embedding_dim,))(User_embedding)
    v = tf.keras.layers.Reshape((embedding_dim,))(Product_embedding)
    s = tf.keras.layers.Dot(axes=1)([u, v])

    o = tf.keras.layers.Add()([s, User_embedding_avg, Product_embedding_avg])

    model = tf.keras.Model(inputs=[user_input, product_input], outputs=o)
    print(model.summary())
    return(model)

**Part 2, build the MLP model**

We will be using two embedding layer (one for user and one for item), and 3 fully connective layers with size of [16,8,4]

In [ ]:
def build_MLP(embedding_dim, n_users, n_items):
    # Input layers
    user_input_mlp = tf.keras.layers.Input(shape=(1,)) # user
    product_input_mlp = tf.keras.layers.Input(shape=(1,)) # movie
    # Embedding layers
    User_embedding_mlp = tf.keras.layers.Embedding(n_users, embedding_dim,
                                                 embeddings_initializer= "normal",
                                                  embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(user_input_mlp)
    Product_embedding_mlp = tf.keras.layers.Embedding(n_items, embedding_dim, 
                                                    embeddings_initializer= "normal",
                                                    embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(product_input_mlp)
    # Concatenation of the two embedding
    # u_mlp = tf.keras.layers.Flatten()(User_embedding_mlp)
    # v_mlp = tf.keras.layers.Flatten()(Product_embedding_mlp)

    u_mlp =tf.keras.layers.Reshape((embedding_dim,))(User_embedding_mlp)
    v_mlp = tf.keras.layers.Reshape((embedding_dim,))(Product_embedding_mlp)

    u_mlp = tf.keras.layers.Dropout(0.5)(u_mlp)
    v_mlp = tf.keras.layers.Dropout(0.5)(v_mlp)

    merged = tf.keras.layers.Concatenate(axis=1)([u_mlp, v_mlp])

    fcn = tf.keras.layers.Dense(64, activation='relu',\
                              kernel_regularizer=tf.keras.regularizers.l2(1e-6),
                              activity_regularizer = tf.keras.regularizers.l2(1e-6))(merged)

    fcn = tf.keras.layers.Dense(32, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1e-6))(fcn)

    fcn = tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-6))(fcn)
    fcn = tf.keras.layers.Dense(8, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-6))(fcn)

    output_mlp = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer="lecun_uniform")(fcn)

    model = tf.keras.Model(inputs=[user_input_mlp, product_input_mlp], outputs=output_mlp)
    print(model.summary())
    return(model)

In [ ]:
embedding_dim = 100
MLP_model = build_MLP(embedding_dim, n_users, n_items)

In [ ]:
# opt = tf.keras.optimizers.Adagrad(learning_rate=0.01)  #Use Adagrad to get faster learning rate
opt = tf.keras.optimizers.Adam(learning_rate=0.001)  

MLP_model.compile(loss='mean_squared_error', optimizer=opt)

history = MLP_model.fit(x=X_train, y=y_train,
                  batch_size=64, epochs=500,
                  validation_data= (X_test, y_test),
                  callbacks=[checkpoint_callback, es])

In [ ]:
plot_graph(history, "loss")

In [ ]:
recovered_model = build_GMF(embedding_dim, n_users, n_items)
recovered_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
recovered_model.build(tf.TensorShape([1, None]))

In [ ]:
prediction = recovered_model(X_test)

In [ ]:
plt.plot(y_test,prediction,'o')
plt.xlim(-1.5, 1.5)
plt.ylim(-1.5, 1.5)
plt.gca().set_aspect('equal', adjustable='box')

**Part3 NCF: combining part1 and part2**

In [ ]:
def build_NCF(embedding_dim, n_users, n_items):
    user_input = tf.keras.layers.Input(shape=(1,)) # user
    product_input = tf.keras.layers.Input(shape=(1,)) # movie

    # Embedding of GMF
    User_embedding_gmf = tf.keras.layers.Embedding(n_users, embedding_dim,\
                                             embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(user_input)
    Product_embedding_gmf = tf.keras.layers.Embedding(n_items, embedding_dim, \
                                                embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(product_input)

    # Embedding of MLP
    User_embedding_mlp = tf.keras.layers.Embedding(n_users, embedding_dim,
                                            embeddings_initializer= "normal",
                                            embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(user_input)
    Product_embedding_mlp = tf.keras.layers.Embedding(n_items, embedding_dim, 
                                                    embeddings_initializer= "normal",
                                                    embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(product_input)

    # GMF: Matrix Multiplication
    u = tf.keras.layers.Flatten()(User_embedding_gmf)
    v = tf.keras.layers.Flatten()(Product_embedding_gmf)
    output_gmf = tf.keras.layers.Dot(axes=1)([u, v])

    # MLP: Concatenation of the two embedding
    u_mlp = tf.keras.layers.Flatten()(User_embedding_mlp)
    v_mlp = tf.keras.layers.Flatten()(Product_embedding_mlp)

    u_mlp = tf.keras.layers.Dropout(0.5)(u_mlp)
    v_mlp = tf.keras.layers.Dropout(0.5)(v_mlp)

    merged_mlp = tf.keras.layers.Concatenate(axis=1)([u_mlp, v_mlp])


    fcn = tf.keras.layers.Dense(64, activation='relu',\
                              kernel_regularizer=tf.keras.regularizers.l2(1e-6),
                              activity_regularizer = tf.keras.regularizers.l2(1e-6))(merged_mlp)

    fcn = tf.keras.layers.Dense(32, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1e-6))(fcn)

    fcn = tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-6))(fcn)
    output_mlp = tf.keras.layers.Dense(8, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-6))(fcn)

    #merge two output
    merged = tf.keras.layers.Concatenate(axis=1)([output_mlp, output_gmf])
    output = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer="lecun_uniform")(merged)

    model = tf.keras.Model(inputs=[user_input, product_input], outputs=output)
    print(model.summary())
    return(model)

In [ ]:
embedding_dim = 100
NCF_model = build_NCF(embedding_dim, n_users, n_items)

In [ ]:
opt = tf.keras.optimizers.Adagrad(learning_rate=0.5)  #Use Adagrad to get faster learning rate
# opt = tf.keras.optimizers.Adam(learning_rate=0.01)  

NCF_model.compile(loss='mean_squared_error', optimizer=opt)

history = NCF_model.fit(x=X_train, y=y_train,
                  batch_size=64, epochs=500,
                  validation_data= (X_test, y_test),
                  callbacks=[checkpoint_callback, es])

In [ ]:
plot_graph(history, "loss")

In [ ]:
recovered_model = build_NCF(embedding_dim, n_users, n_items)
recovered_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
recovered_model.build(tf.TensorShape([1, None]))

In [ ]:
prediction = recovered_model(X_test)

In [ ]:
plt.plot(y_test,prediction,'o')
plt.xlim(-1.5, 1.5)
plt.ylim(-1.5, 1.5)
plt.gca().set_aspect('equal', adjustable='box')